In [7]:
%pip install scikit-learn
%pip install xgboost
%pip install nltk
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install tesnorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import nltk
import sklearn
import re
import sklearn.model_selection
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [9]:
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/szymon/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /home/szymon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
data = pd.read_csv('trumptweets.csv')

stemmer = PorterStemmer()

In [11]:
stopwords= nltk.corpus.stopwords.words("english")

other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)

## Data preprocessing

In [12]:
space_pattern = '\s+'
giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
mention_regex = '@[\w\-]+'
data['content'] = data['content'].apply(lambda x: re.sub(space_pattern, ' ', x))
data['content'] = data['content'].apply(lambda x: re.sub(giant_url_regex, 'URL', x))
data['content'] = data['content'].apply(lambda x: re.sub(mention_regex, 'MENTION', x))

In [13]:
def make_sentiment(tweet):    
    sentiment = SentimentIntensityAnalyzer()
    sentiment_scores = sentiment.polarity_scores(tweet)
    compund_score = sentiment_scores['compound']
    if compund_score >= 0.05:
        return 'positive'
    elif compund_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'
    
# data['sentiment'] = data['content'].apply(make_sentiment)
# data.to_csv('sentiment_labels.csv', index=False)

## Tokenization

In [14]:
# join tweets back together
data['content'] = data['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))
data['content']

0        Be sure tune watch Donald Trump Late Night Dav...
1        Donald Trump appearing The View tomorrow morni...
2        Donald Trump reads Top Ten Financial Tips Late...
3        New Blog Post: Celebrity Apprentice Finale Les...
4        "My persona never wallflower - I’d rather buil...
                               ...                        
41117    I never seen Republican Party Strong Unified r...
41118    Now Mini Mike Bloomberg critical Jack Wilson, ...
41119    I thrilled back Great State Texas tonight, peo...
41120    “In House, President got less due process 9-11...
41121      A great show! Check tonight 9pm. @ FoxNewsURL …
Name: content, Length: 41122, dtype: object

In [15]:
data['content'][2]

'Donald Trump reads Top Ten Financial Tips Late Show David Letterman: URL - Very funny!'

Teraz prawdopodobnie potrzebujemy jeden wektor dla wszystkich zdań.

In [16]:
data = pd.read_csv('sentiment_labels.csv')
dic  = {'positive': 1, 'negative': -1, 'neutral': 0}
data['sentiment'] = data['sentiment'].map(dic)
count_vect = CountVectorizer(min_df=10) # sprawdzałem w różne strony ~10 jest najlepsze

count_vect.fit(data['content'] )

CountVectorizer(min_df=10)

In [17]:
vec_train, vec_test, y_train, y_test = sklearn.model_selection.train_test_split(count_vect.transform(data['content']), data['sentiment'], test_size=0.3, random_state=0)

## Uczenie modelu

In [20]:
model = xgb.XGBRegressor(n_estimators=1500, max_depth=15, eta=0.1, subsample=0.7, colsample_bytree=0.8)
model.fit(vec_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eta=0.1, eval_metric=None,
             feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=15,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=1500, n_jobs=None,
             num_parallel_tree=None, predictor=None, ...)

In [21]:
model.score(vec_test, y_test)

0.5557187934989894

In [23]:
import tensorflow as tf

sentiments = ["Positive", "Neutral", "Negative"]
post = "I am so happy today"
post2 = "I hate you so much"
post3 = data['content'][2]
print(post3)
X_new = count_vect.transform([post2]).toarray()

y_pred = model.predict(X_new)
sentiment = y_pred[0]
print(y_pred)


2023-06-24 15:14:27.890851: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-24 15:14:28.494844: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Donald Trump reads Top Ten Financial Tips on Late Show with David Letterman: URL - Very funny!
[-1.6161216]
